In [14]:
import pandas as pd
import gzip

df = pd.read_csv('Real_mutated_seqs.csv')

with gzip.open('library/library.fasta.gz', 'wt') as fasta_file:
    for idx, row in df.iterrows():
        fasta_file.write(f">seq{idx+1}|GENE{idx+1}|source\n")
        fasta_file.write(f"{row['Sequence']}\n")


[sbPCR](https://github.com/wendao/sbPCR) is a machine learning model that can predict the reactivity of cysteines based on just the sequence. The accompanying paper by Wang et al. can be found [here](https://pubs.acs.org/doi/10.1021/acs.biochem.7b00897).
Please note that you need `libsvm` installed in order to use sbPCR.

In [4]:
%%bash
git clone https://github.com/wendao/sbPCR

Cloning into 'sbPCR'...


We clear the sbPCR_run directory from previous results and extract the contents of our sequence library. We then pass the sequences into sbPCR as outlined in the documentation. Please note that the "Accuracy" prediction in the output is an artifact of `libsmv`and has no bearing on this task.

In [15]:
%%bash
rm -rf sbPCR_run && mkdir sbPCR_run
gunzip -c library/library.fasta.gz > library/library.fasta

cd sbPCR_run
python ../sbPCR/scripts/get_align.py  ../library/library.fasta
python ../sbPCR/scripts/get_feature.py
svm-predict  formated_input  ../sbPCR/models/train_v1.model  predict

Accuracy = 96.9697% (64/66) (classification)


We can now extract the results from the sbPCR run. We calculate the number of sequences deemed reactive by sbPCR as a percentage of the whole set, and store it in `filtered_sequences.fasta`. On average, about 10% of sequences are deemed reactive by sbPCR. Notably, the original wild type is NOT deemed reactive. sbPCR judges reactivity based on empirical data, in which the top 10% most reactive samples received the label "reactive".

In [16]:
import pandas as pd, pathlib
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO

wd = pathlib.Path("sbPCR_run")

# 1 ─ read sbPCR outputs (label only)
align = pd.read_csv(
    wd / "align",  sep=r"\s+",  header=None,
    names=["variant", "pos", "window"]
)
pred  = pd.read_csv(
    wd / "predict", sep=r"\s+", header=None,
    names=["label"], dtype={"label": int}
)

data = pd.concat([align, pred], axis=1)

# 2 ─ is WT reactive at both catalytic sites?
wt_rows = data[data.variant.str.startswith("WT") & data.pos.isin([12, 15])]
wt_reactive = (len(wt_rows) == 2) and (wt_rows["label"] == 1).all()
print("WT reactive at both cysteines = ", wt_reactive)

# 3 ─ variants with *both* sites reactive
both_hits = (
    data.query("pos in [12, 15] and label == 1")
        .groupby("variant")
        .size()
        .loc[lambda s: s == 2]
        .index
)

n_hits = len(both_hits)
lib_size = data.variant.nunique()
pct = n_hits / lib_size * 100

print(f"{n_hits}/{lib_size} = {pct:.2f}% variants have both Cys 12 and Cys 15 predicted reactive")

# 4 ─ Load full-length sequences and map VAR IDs

# Load full sequences as dictionary: {full_id : SeqRecord}
full_sequences = SeqIO.to_dict(SeqIO.parse("library/library.fasta", "fasta"))

# Build mapping: {VARxxxxx : SeqRecord}
id_map = {}
for full_id, record in full_sequences.items():
    parts = full_id.split("|")
    if len(parts) >= 2:
        var_id = parts[1]  # e.g. VAR00020
        id_map[var_id] = record

# 5 ─ Extract filtered variants
filtered = data[data.variant.isin(both_hits)].drop_duplicates("variant")

# 6 ─ Build SeqRecords using full-length sequences

# First, include the wild-type
wt_record = None
for full_id, record in full_sequences.items():
    if full_id.startswith("WT|WT|WT"):
        wt_record = SeqRecord(record.seq, id="WT", description="wild-type")
        break

# Collect records: WT first, then variants
records = []

if wt_record:
    records.append(wt_record)

# Add variants
records.extend([
    SeqRecord(
        id_map[row.variant].seq,
        id=row.variant,
        description=""
    )
    for row in filtered.itertuples()
])

# 7 ─ Write full-length filtered sequences
SeqIO.write(records, "filtered_fastas/filtered_sequences.fasta", "fasta")


WT reactive at both cysteines =  False
0/21 = 0.00% variants have both Cys 12 and Cys 15 predicted reactive


0

In [1]:
#!/usr/bin/env python3
"""
Batch “CIF → PROPKA” runner for the alphafold/ directory.
Focuses on CYS 12 and CYS 15 only.

Output
------
• Wild-type (3IWL) pKa values
• Mean and s.d. for each site across all variants
• Top-10 variants with the largest |ΔpKa| at either site
"""

from pathlib import Path
import subprocess
import statistics as stats
import tempfile
import re
import gemmi
import shutil
import sys

# ────────────────────────── USER SETTINGS ──────────────────────────
folder           = Path('alphafold')     # directory containing *.cif
wild_type_file   = '3iwl.cif'            # case-exact filename
propka_exe       = 'propka3'             # command in $PATH
sites_of_interest = {12, 15}             # residue numbers
# ───────────────────────────────────────────────────────────────────

# Regex that matches a normal PROPKA Cys line (with or without leading index)
cys_line = re.compile(
    r'^\s*(?:\d+\s+)?'        # optional index
    r'CYS\s+'
    r'(\d+)\s+'               # residue number   – group 1
    r'([A-Za-z0-9]?)\s+'      # chain ID (0–1 ch) – group 2
    r'([0-9]+\.[0-9]+)'       # pKa value         – group 3
)

def extract_cys_pka(pka_file: Path) -> dict[int, float]:
    """Return {resnum: pKa} for sites_of_interest in one PROPKA .pka file."""
    result: dict[int, float] = {}
    with pka_file.open() as fh:
        for line in fh:
            m = cys_line.match(line)
            if not m:
                continue
            resnum = int(m.group(1))
            if resnum not in sites_of_interest:
                continue
            pka = float(m.group(3))
            if 0.1 < pka < 99.0 and resnum not in result:  # first non-artefact wins
                result[resnum] = pka
    return result

def run_one_cif(cif_path: Path, workdir: Path) -> dict[int, float]:
    """Convert CIF→PDB, run PROPKA in *workdir*, return pKa dict."""
    pdb_path = workdir / (cif_path.stem + '.pdb')
    # CIF → PDB (Gemmi)
    structure = gemmi.make_structure_from_block(
        gemmi.cif.read_file(str(cif_path)).sole_block()
    )
    structure.write_pdb(str(pdb_path))

    # PROPKA (run inside workdir so output lands there)
    subprocess.run([propka_exe, pdb_path.name],
                   cwd=workdir,
                   stdout=subprocess.DEVNULL,
                   stderr=subprocess.DEVNULL,
                   check=True)

    pka_file = pdb_path.with_suffix('.pka')
    if not pka_file.is_file():
        raise FileNotFoundError(f'Expected {pka_file} not written by PROPKA')

    return extract_cys_pka(pka_file)

# ────────────────────────── MAIN WORK ─────────────────────────────
if not folder.is_dir():
    sys.exit(f'Error: directory {folder} does not exist')

records: dict[str, dict[int, float]] = {}   # {variant_name: {resnum: pKa}}

with tempfile.TemporaryDirectory() as tmp:
    workdir = Path(tmp)

    for cif in sorted(folder.glob('*.cif')):
        try:
            pka_dict = run_one_cif(cif, workdir)
            if pka_dict:            # store only if at least one site was found
                records[cif.stem] = pka_dict
        except Exception as exc:
            print(f'[WARN] {cif.name}: {exc}', file=sys.stderr)

# ensure wild-type is present
wt_name = Path(wild_type_file).stem
if wt_name not in records:
    sys.exit(f'Wild-type file {wild_type_file} did not yield data – aborting.')

wt = records[wt_name]

# ────────────────────────── STATISTICS ────────────────────────────
stats_summary = {site: [] for site in sites_of_interest}
for rec in records.values():
    for site in sites_of_interest:
        if site in rec:
            stats_summary[site].append(rec[site])

# rank variants by maximum |ΔpKa| at the two sites
ranked: list[tuple[float, str, dict[int, float]]] = []
for name, rec in records.items():
    if name == wt_name:
        continue
    delta = max(
        abs(rec.get(site, wt[site]) - wt[site])
        for site in sites_of_interest
        if site in wt
    )
    ranked.append((delta, name, rec))

ranked.sort(reverse=True)
top10 = ranked[:10]

# ────────────────────────── REPORT ───────────────────────────────
print('\nWild-type pKa values (3IWL)')
for site in sorted(sites_of_interest):
    if site in wt:
        print(f'  CYS {site:>4}: {wt[site]:.2f}')
    else:
        print(f'  CYS {site:>4}: not titratable (disulfide)')

print('\nSummary statistics across all variants')
for site in sorted(sites_of_interest):
    values = stats_summary[site]
    if values:
        mean = stats.mean(values)
        sd   = stats.stdev(values) if len(values) > 1 else 0.0
        print(f'  CYS {site:>4}: mean = {mean:.2f}, sd = {sd:.2f}, n = {len(values)}')
    else:
        print(f'  CYS {site:>4}: no data')

print('\nTop 10 variants by |ΔpKa| (largest site difference)')
for delta, name, rec in top10:
    parts = []
    for site in sorted(sites_of_interest):
        if site in rec and site in wt:
            diff = rec[site] - wt[site]
            parts.append(f'CYS {site} {rec[site]:.2f} (Δ{diff:+.2f})')
    print(f'  {name:25}  {"; ".join(parts)}')



Wild-type pKa values (3IWL)
  CYS   12: 9.25
  CYS   15: 6.71

Summary statistics across all variants
  CYS   12: mean = 9.04, sd = 0.70, n = 21
  CYS   15: mean = 7.00, sd = 1.34, n = 20

Top 10 variants by |ΔpKa| (largest site difference)
  fold_fold_job_19_model_0   CYS 12 7.64 (Δ-1.61); CYS 15 10.08 (Δ+3.37)
  fold_1_model_0             CYS 12 7.53 (Δ-1.72); CYS 15 10.07 (Δ+3.36)
  fold_2_model_0             CYS 12 7.75 (Δ-1.50); CYS 15 9.77 (Δ+3.06)
  fold_fold_job_20_model_0   CYS 12 8.32 (Δ-0.93)
  fold_fold_job_21_model_0   CYS 12 8.37 (Δ-0.88)
  fold_5_model_0             CYS 12 9.19 (Δ-0.06); CYS 15 5.83 (Δ-0.88)
  fold_fold_job_11_model_0   CYS 12 9.42 (Δ+0.17); CYS 15 5.99 (Δ-0.72)
  fold_fold_job_15_model_0   CYS 12 9.25 (Δ+0.00); CYS 15 6.00 (Δ-0.71)
  fold_fold_job_13_model_0   CYS 12 9.66 (Δ+0.41); CYS 15 6.05 (Δ-0.66)
  fold_8_model_0             CYS 12 9.17 (Δ-0.08); CYS 15 6.05 (Δ-0.66)
